In [38]:
import chromadb
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
import os
import ast
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk

### Análisis exploratorio

In [3]:
dn = pd.read_csv('../data/processed/Daniel_Noboa_preprocesado.csv')

In [4]:
dn

,Id,Candidato,Temas tratados,Descripción,Entrevista
0,DNE001,Daniel Noboa,formacion trayectori segur ciudadan cooper int...,entrev president daniel nobo abord divers tem ...,hol bienven dialog electoral ug oliv merchan v...
1,DNE002,Daniel Noboa,segur justici desaparicion cuatr joven guayaqu...,entrev president daniel nobo radi democraci ab...,unid nacional pro paz segur ecuador revist opi...
2,DNE003,Daniel Noboa,gobiern polit democraci desafi rol vot inform ...,conversatori abord tem clav democraci particip...,alo buen dias favor tom asient graci preci des...
3,DNP004,Daniel Noboa,reduccion pobrez desiguald acces universal sal...,document present plan integral desarroll ecuad...,egur desarroll bienest ecuatorian mejor sistem...
4,DNP005,Daniel Noboa,desarroll econom sosten transform product inno...,document present plan integral desarroll econo...,desarroll econom sosten mejor calid vid ecuato...
5,DNP006,Daniel Noboa,medi ambient energ conect conserv ambiental ad...,document present enfoqu integral desarroll sos...,ecuador conect prepar futur ciudad resilient e...
6,DNP007,Daniel Noboa,institucional public acces inform particip ciu...,document present plan transform ecuatorian cen...,nuev nuev ecuador eficient transparent partici...


### Unificación del corpus

Tenemos dos carpetas que contienen el corpus:

1. raw: Contiene los archivos sin procesar del corpus
2. processed: Contiene los archivos preprocesados del corpus

Debido a que se ha utilizado lematización al preprocesar el texto, este tiene la característica de recortar palabras, por lo que al recuperar los documentos que queremos no podemos mostrar eso al usuario, es por esto que se hará lo siguiente:

1. Concatenar todos los archivos de cada carpeta respectivamente, el resultado será dos archivos (raw_corpus.csv y processed_corpus.csv)
2. Se realizará un join entre estos dos archivos utilizando la columna 'ID' para esta unión.

In [6]:
base_dir = r'../data/raw'
processed_files = os.scandir(base_dir)
unified_raw_df = pd.DataFrame()

for file in processed_files:
    df = pd.read_csv(rf'{base_dir}/{file.name}')
    unified_raw_df = pd.concat([df, unified_raw_df])
    
del df

In [7]:
base_dir = r'../data/processed'
processed_files = os.scandir(base_dir)
unified_processed_df = pd.DataFrame()

for file in processed_files:
    df = pd.read_csv(rf'{base_dir}/{file.name}')
    unified_processed_df = pd.concat([df, unified_processed_df])

In [ ]:
unified_raw_df['Id'] = unified_raw_df['Id'].astype(str)
unified_processed_df['Id'] = unified_processed_df['Id'].astype(str)

In [17]:
unified_global_df = pd.merge(unified_raw_df, unified_processed_df, on='Id', suffixes=('_raw', '_pre'))

In [22]:
unified_global_df.drop(columns = ['Descripción_pre', 'Temas tratados_pre', 'Candidato_pre'], inplace=True)
unified_global_df

,Id,Candidato_raw,Temas tratados_raw,Descripción_raw,Entrevista_raw,Entrevista_pre
0,WGE001,Wilson Gomez,Conformación del binomio presidencial; Relació...,El documento es una entrevista en la que un ca...,Yo vengo trabajando para muchos presidentes he...,veng trabaj president oportun serv voluntari e...
1,WGE002,Wilson Gomez,Crisis del país; Propuestas de rescate y trans...,El documento expone la visión y propuestas del...,Buenos días nos encontramos con Wilson Gómez c...,buen dias encontr wilson gomez candidat presid...
2,WGE003,Wilson Gomez,La necesidad de un cambio en el modelo educati...,El documento es una entrevista con el candidat...,un gran abrazo para la inmensa audiencia de ra...,gran abraz inmens audienci radi moren ecuador ...
3,WGP004,Wilson Gomez,"El rol de la agricultura, la pesca y la indust...",El documento analiza el sector agropecuario y ...,Parte 1: Diagnóstico de la Situación Actual La...,part diagnost situacion actual agricultur pesc...
4,WGP005,Wilson Gomez,Extracción de petróleo; Historia y auge de la ...,El documento analiza tres sectores clave en Ec...,Parte 1: Extracción de petróleo en Ecuador Dia...,part extraccion petrole ecuador diagnost situa...
5,WGP006,Wilson Gomez,Definición y módulos de FinTech; Criptomonedas...,"El documento analiza el sector FinTech, su imp...",Parte 1: ¿Qué es FinTech y cuáles son sus módu...,part fintech cual modul principal termin finte...
6,WGP007,Wilson Gomez,Economía agraria; Industrialización y auge pet...,El texto describe la evolución económica de Ec...,EXTRACCIÓN DE ECONOMÍA & PRODUCTIVIDAD Histori...,extraccion econom product histori econom ecuad...
7,WGP008,Wilson Gomez,Ecoturismo en Ecuador; Economía y Productivida...,El documento propone un plan de desarrollo par...,Estrategia para el Desarrollo del Ecoturismo e...,estrategi desarroll ecotur ecuador problem afe...
8,DNE001,Daniel Noboa,"Formación y trayectoria, Seguridad ciudadana, ...",La entrevista con el presidente Daniel Noboa a...,Hola a todos Bienvenido a diálogo electoral ug...,hol bienven dialog electoral ug oliv merchan v...
9,DNE002,Daniel Noboa,Seguridad y Justicia; Desaparición de los cuat...,La entrevista con el presidente Daniel Noboa e...,la unidad nacional en pro de la paz y la segur...,unid nacional pro paz segur ecuador revist opi...


In [26]:
unified_raw_df.to_csv(rf'../data/raw/raw_corpus.csv', index=False)
unified_processed_df.to_csv(rf'../data/processed/processed_corpus.csv', index=False)
unified_global_df.to_csv(rf'../data/embeddings/unified_corpus.csv', index=False)

### Generación de embeddings

Una vez tenemos todo el corpus unificado en un único archivo, tenemos que calcular los embeddings para el contenido de cada entrevista, esto lo haremos utilizando la libreria SentenceTransformer.

In [19]:
unified_corpus = pd.read_csv('../data/embeddings/unified_corpus.csv')

In [29]:
# Modelo con el que vamos a generar embeddings
modelo = SentenceTransformer("all-MiniLM-L6-v2")

# Generación de los embeddings a partir de corpus preprocesado
unified_corpus['embedding_entrevista'] = unified_corpus['Entrevista_pre'].apply(lambda x: modelo.encode(x).tolist())

In [30]:
unified_corpus['embedding_entrevista']

0     [-0.014551389962434769, -0.0037382026202976704...
1     [0.007717538625001907, -0.04105502739548683, -...
2     [-0.007631767075508833, -0.03830636292695999, ...
3     [0.01490846835076809, -0.08086168766021729, -0...
4     [0.013256512582302094, -0.06549493223428726, -...
5     [-0.008189497515559196, -0.02202538400888443, ...
6     [0.020869851112365723, -0.09945572167634964, 0...
7     [0.06572176516056061, -0.0940171629190445, -0....
8     [0.006839875131845474, -0.03802037984132767, 0...
9     [0.016809863969683647, -0.010982906445860863, ...
10    [-0.05267263576388359, 0.05301240086555481, -0...
11    [0.011799047701060772, -0.05923914164304733, -...
12    [-0.0010314254323020577, -0.06436630338430405,...
13    [0.08150909096002579, -0.052145641297101974, 0...
14    [0.029494814574718475, -0.025782305747270584, ...
Name: embedding_entrevista, dtype: object

In [31]:
unified_corpus.to_csv(rf'../data/embeddings/unified_corpus_embeddings.csv', index=False)

### Base de datos vectorial con ChromaDB

Una vez tenemos un único archivo que contiene todo el corpus junto a los embeddings del contenido de cada entrevista, vamos a construir una base de datos vectorial con ChromaDB para la realización de consultas.

La estructura a manejar será la siguiente:
- ID: El identificador alfanumérico de cada entrevista
- embeddings: El vector representativo del contenido de la entrevista
- metadatos: El nombre del candidato, tema tratado y una descripción de la entrevista

In [32]:
# Creación de la base de datos vectorial
chroma_client = chromadb.PersistentClient(path="../db/")
collection = chroma_client.get_or_create_collection("entrevistas")

In [33]:
embeddings_df = pd.read_csv('../data/embeddings/unified_corpus_embeddings.csv')

In [34]:
embeddings_df['embedding_entrevista']

0     [-0.014551389962434769, -0.0037382026202976704...
1     [0.007717538625001907, -0.04105502739548683, -...
2     [-0.007631767075508833, -0.03830636292695999, ...
3     [0.01490846835076809, -0.08086168766021729, -0...
4     [0.013256512582302094, -0.06549493223428726, -...
5     [-0.008189497515559196, -0.02202538400888443, ...
6     [0.020869851112365723, -0.09945572167634964, 0...
7     [0.06572176516056061, -0.0940171629190445, -0....
8     [0.006839875131845474, -0.03802037984132767, 0...
9     [0.016809863969683647, -0.010982906445860863, ...
10    [-0.05267263576388359, 0.05301240086555481, -0...
11    [0.011799047701060772, -0.05923914164304733, -...
12    [-0.0010314254323020577, -0.06436630338430405,...
13    [0.08150909096002579, -0.052145641297101974, 0...
14    [0.029494814574718475, -0.025782305747270584, ...
Name: embedding_entrevista, dtype: object

In [35]:
for index, row in embeddings_df.iterrows():
    collection.add(
        ids = [row['Id']],
        documents = [row['Entrevista_raw']],
        embeddings = [ast.literal_eval(row['embedding_entrevista'])],
        metadatas = [{
            'candidato': row['Candidato_raw'],
            'temas_tratados': row['Temas tratados_raw'],
            'descripcion': row['Descripción_raw']
        }]
    )

### Realizar una query

In [41]:
# Inicializar el stemmer
stemmer = PorterStemmer()

# Cargar stopwords en español
stop_words = set(stopwords.words('spanish'))

# Función de preprocesamiento (tokenización + stopwords + stemming)
def preprocesar_texto(texto):
    tokens = nltk.word_tokenize(texto.lower())
    tokens_sin_stopwords = [t for t in tokens if t not in stop_words]
    tokens_stemm = [stemmer.stem(t) for t in tokens_sin_stopwords]
    return " ".join(tokens_stemm)

In [44]:
query = 'Vamos a reducir los asaltos en el país'

# Preprocesar la consulta
preproc_query = preprocesar_texto(query)

# Generar el embedding de la consulta
embedding_query = modelo.encode(preproc_query)

In [66]:
# Realizar la consulta en la base de datos
resultados = collection.query(query_embeddings=[embedding_query], n_results=3)  # Los 3 resultados más cercanos

# Mostrar los resultados
print("Resultados de la consulta:")
print()
for i in range(len(resultados['documents'][0])):
    print(f"Resultado {i+1}")
    print(f"ID: {resultados['ids'][0][i]}")
    print(f"Contenido: {resultados['documents'][0][i]}")
    print(f"Metadatos: {resultados['metadatas'][0][i]}")
    print("")

Resultados de la consulta:

Resultado 1
ID: WGE003
Contenido: un gran abrazo para la inmensa audiencia de radio Morena en todo el Ecuador y en varias partes desde el mar hasta las montañas delar del carch al macará tras las fronteras radio moneda llega a la entrada de Lima con los 30,000 watios de potencia que tiene Eh bueno ya tenemos en nuestros estudios a Enrique Gómez él es El reemplazo de J topic una persona vinculada al resao a las ayudas solidarias en Guayaquil ha hecho mucho en el cuerpo de bomberos siempre y decirle hace un rato entrevistamos a un de los encuestadores digo yo que se en lugar de continuar y al menos mantener lo que existía por esta falta de inversión No se ha dado lo primero debe ser no quitarle el dinero a las empresas públicas qué es lo que pasa hoy el gobierno central quita el dinero de las empresas públicas que debería reinvertir en los mantenimientos y en las nuevas inversiones y lo que termina usando es el gasto de pago de sueldos o el pago de otros proye

In [55]:
print(resultados.keys())

dict_keys(['ids', 'embeddings', 'documents', 'uris', 'data', 'metadatas', 'distances', 'included'])
